In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define constants
DATASET_PATH = "birds_audio_dataset"  # Path to your dataset folder
SR = 16000  # Sample rate
DURATION = 5  # Duration of audio clips in seconds
NUM_CLASSES = len(os.listdir(DATASET_PATH))  # Number of bird species

# Function to load audio file and convert to spectrogram
def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=SR, duration=DURATION, mono=True)
    # Convert audio to spectrogram
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=SR)
    return spectrogram

# Function to preprocess dataset
def preprocess_dataset(dataset_path):
    X = []
    y = []
    for class_idx, class_label in enumerate(os.listdir(dataset_path)):
        class_path = os.path.join(dataset_path, class_label)
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            # Load audio and convert to spectrogram
            spectrogram = load_audio(file_path)
            X.append(spectrogram)
            y.append(class_idx)
    return np.array(X), np.array(y)

# Load and preprocess dataset
X, y = preprocess_dataset(DATASET_PATH)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build WaveNet model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Reshape((X_train.shape[1], X_train.shape[2], 1)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=16)

# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test Accuracy:", test_accuracy)

model.save("birds_classifier_model.h5")
